In [ ]:
import numpy as np

import section3_1 as ld

import section4_7_a as hs_oil
import section4_7_b as hs_gas
import section4_7_c as hs_eheater
import section4_7_d as hs_ehpump
import section4_7_e as hs_gas_hybrid
import section4_7_f as hs_hybrid_gas
import section4_7_g as hs_whybrid

import section4_7_j as rad_panel
import section4_7_k as rad_fanc
import section4_7_l as rad_floor

import section4_7_i as pipe

from section4_7_common import get_Q_out_H_hs_d_t

from section11_1 import \
    load_outdoor, \
    get_Theta_ex, \
    get_X_ex, \
    get_h_ex

## 第七節　温水暖房

1．適用範囲～4．記号及び単位は省略

### 5. 最大暖房出力

省略

In [ ]:
# 最大暖房出力
def get_Q_max_H_d_t_i(radiator, A_HCZ, Theta_SW, region, mode, R_type):
    return get_Q_max_H_rad_d_t_i(radiator, A_HCZ, Theta_SW, region, mode, R_type)

### 6. エネルギー消費量

#### 6.1 消費電力量

$$
\large
E_{E,H,d,t} = E_{E,hs,d,t} + \sum_{i} (E_{E,rad,d,t,i})
$$

$E_{E,H,d,t}$：日付$d$の時刻$t$における1時間当たりの消費電力量(kWh/h)  
$E_{E,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの温水暖房用熱源機の消費電力量(kWh/h)  
$E_{E,rad,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された放熱器の消費電力量(kWh/h)    

In [ ]:
# 消費電力量 (1)
def get_E_E_H_d_t(H_HS, H_MR, H_OR, A_A, A_MR, A_OR, region, mode_MR, mode_OR, L_T_H_rad, CG=None):
    rad_types = get_rad_type_list()

    rad_list = get_rad_list(H_MR, H_OR)

    E_E_hs_d_t = get_E_E_hs_d_t(H_HS, H_MR, H_OR, region, A_A, A_MR, A_OR, mode_MR, mode_OR, L_T_H_rad, CG)

    E_E_rad_d_t = np.zeros((5, 24 * 365))
    for i in [1, 3, 4, 5]:
        if rad_list[i - 1] is None:
            continue
        if rad_list[i - 1]['type'] in rad_types:
            radiator = rad_list[i - 1]
            R_type = '主たる居室' if i == 1 else 'その他の居室'
            mode = mode_MR if i == 1 else mode_OR
            A_HCZ = get_A_HCZ_i(i, A_A, A_MR, A_OR)

            Theta_SW_hs_op = get_Theta_SW_hs_op(H_HS['type'], CG['CG_category'] if CG is not None else None)
            p_hs_d_t = get_p_hs_d_t(Theta_SW_hs_op, rad_list, L_T_H_rad, A_A, A_MR, A_OR, region, mode_MR, mode_OR)
            Theta_SW_d_t = get_Theta_SW_d_t(Theta_SW_hs_op, p_hs_d_t)
            Q_max_H_rad_d_t_i = get_Q_max_H_rad_d_t_i(radiator, A_HCZ, Theta_SW_d_t, region, mode, R_type)
            Q_T_H_rad_d_t_i = get_Q_T_H_rad_d_t_i(Q_max_H_rad_d_t_i, L_T_H_rad[i - 1])

            E_E_rad_d_t_i = get_E_E_rad_d_t_i(i, radiator, Q_T_H_rad_d_t_i, Theta_SW_d_t, A_A, A_MR, A_OR, region, mode,
                                              R_type)
            E_E_rad_d_t[i - 1, :] = E_E_rad_d_t_i

            print('{} E_E_rad_d_t_{} = {} [KWh] (L_T_H_rad_d_t_{} = {} [MJ])'.format(radiator['type'], i,
                                                                                     np.sum(E_E_rad_d_t_i), i,
                                                                                     np.sum(L_T_H_rad[i - 1])))

    E_E_H_d_t = E_E_hs_d_t + np.sum(E_E_rad_d_t, axis=0)

    return E_E_H_d_t

#### 6.2 灯油消費量

$$
\large
E_{K,H,d,t} = E_{K,hs,d,t}
$$

$E_{K,H,d,t}$：日付$d$の時刻$t$における1時間当たりの灯油消費量(MJ/h)  
$E_{K,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの温水暖房用熱源機の灯油消費量(MJ/h) 

In [ ]:
# 灯油消費量 (2)
def get_E_K_H_d_t(H_HS, H_MR, H_OR, A_A, A_MR, A_OR, region, mode_MR, mode_OR, L_T_H_rad, CG):
    return get_E_K_hs_d_t(H_HS, H_MR, H_OR, A_A, A_MR, A_OR, region, mode_MR, mode_OR, L_T_H_rad, CG)

#### 6.3 ガス消費量

$$
\large
E_{G,H,d,t} = E_{G,hs,d,t}
$$

$E_{G,H,d,t}$：日付$d$の時刻$t$における1時間当たりのガス消費量(MJ/h)  
$E_{G,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの温水暖房用熱源機のガス消費量(MJ/h)  

In [ ]:
# ガス消費量 (3)
def get_E_G_H_d_t(H_HS, H_MR, H_OR, A_A, A_MR, A_OR, region, mode_MR, mode_OR, L_T_H_rad, CG):
    return get_E_G_hs_d_t(H_HS, H_MR, H_OR, A_A, A_MR, A_OR, region, mode_MR, mode_OR, L_T_H_rad, CG)

#### 6.4 その他の燃料による一次エネルギー消費量

$$
\large
E_{M,H,d,t} = E_{M,hs,d,t}
$$

$E_{M,H,d,t}$：日付$d$の時刻$t$における1時間当たりのその他の燃料による一次エネルギー消費量(MJ/h)  
$E_{M,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの温水暖房用熱源機のその他の燃料による一次エネルギー消費量(MJ/h)  

In [ ]:
# その他の燃料による一次エネルギー消費量 (4)
def get_E_M_H_d_t(H_HS):
    return get_E_M_hs_d_t(H_HS)

### 7. 温水暖房用熱源機のエネルギー消費量

#### 7.1 エネルギー消費量

省略

#### 7.2 暖房出力

$$
\large
Q_{out,H,hs,d,t} = min(Q_{dmd,H,hs,d,t}, Q_{max,H,hs,d,t})
$$

$Q_{out,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの温水暖房用熱源機の暖房出力(MJ/h)  
$Q_{max,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの温水暖房用熱源機の最大暖房出力(MJ/h)  
$Q_{dmd,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの温水暖房用熱源機の温水熱需要(MJ/h)  

$$
\large
Q_{dmd,H,hs,d,t} = \sum_{i} (Q_{dmd,H,ln,d,t,i})
$$

$Q_{dmd,H,ln,d,t,i}$：日付$d$の時刻$t$における1時間当たりの放熱系統$i$の温水熱需要(MJ/h)  

In [ ]:
# 温水暖房用熱源機の温水熱需要 (6)
def get_Q_dmd_H_hs_d_t(rad_list, pipe_insulation, underfloor_pipe_insulation, Theta_SW_d_t, A_A, A_MR, A_OR, region,
                       mode_MR, mode_OR,
                       L_T_H_rad_d_t):
    MR_rad_type, r_Af_1 = get_MR_rad_type_and_r_Af_1(rad_list)

    outdoor = load_outdoor()
    Theta_ex = get_Theta_ex(region, outdoor)

    Q_dmd_H_hs_d_t = np.zeros_like(Theta_SW_d_t)

    for i in [1, 3, 4, 5]:

        if rad_list[i - 1] is None:
            continue

        # 1時間当たりの暖冷房区画iに設置された放熱器の最大暖房出力
        A_HCZ = get_A_HCZ_i(i, A_A, A_MR, A_OR)
        R_type = '主たる居室' if i == 1 else 'その他の居室'
        mode = mode_MR if i == 1 else mode_OR
        Q_max_H_rad_d_t_i = get_Q_max_H_rad_d_t_i(rad_list[i - 1], A_HCZ, Theta_SW_d_t, region, mode, R_type)

        # 1時間当たりの暖冷房区画iに設置された放熱器の処理暖房負荷
        Q_T_H_rad_d_t_i = get_Q_T_H_rad_d_t_i(Q_max_H_rad_d_t_i, L_T_H_rad_d_t[i - 1])

        # 1時間当たりの暖冷房区画iに設置された放熱器の温水供給運転率
        r_WS_rad_d_t_i = get_r_WS_rad_d_t_i(
            A_HCZ=A_HCZ,
            radiator=rad_list[i - 1],
            Q_T_H_rad=Q_T_H_rad_d_t_i,
            Theta_SW=Theta_SW_d_t,
            region=region,
            mode=mode,
            R_type=R_type
        )

        Q_dmd_H_ln_d_t_i = get_Q_dmd_H_ln_d_t_i(
            i=i,
            radiator=rad_list[i - 1],
            Q_T_H_rad_d_t_i=Q_T_H_rad_d_t_i,
            Q_max_H_rad_d_t_i=Q_max_H_rad_d_t_i,
            L_T_H_rad_d_t_i=L_T_H_rad_d_t[i - 1],
            Theta_SW_d_t=Theta_SW_d_t,
            Theta_ex=Theta_ex,
            r_WS_rad_d_t_i=r_WS_rad_d_t_i,
            A_A=A_A,
            pipe_insulation=pipe_insulation,
            underfloor_pipe_insulation=underfloor_pipe_insulation,
            MR_rad_type=MR_rad_type,
            r_Af_1=r_Af_1
        )

        Q_dmd_H_hs_d_t = Q_dmd_H_hs_d_t + Q_dmd_H_ln_d_t_i

    return Q_dmd_H_hs_d_t


def get_MR_rad_type_and_r_Af_1(rad_list):
    if rad_list[0] is not None:
        MR_rad_type = rad_list[0]['type']
        if 'r_Af' in rad_list[0]:
            r_Af_1 = rad_list[0]['r_Af']
        else:
            r_Af_1 = None
    else:
        MR_rad_type = None
        r_Af_1 = None
    return MR_rad_type, r_Af_1

#### 7.3 温水供給運転率

$Q_{dmd,H,hs,d,t} > 0$ の場合

$$\large
r_{WS,hs,d,t} = \left\{ \begin{array}{ll}
    r_{WS,ln,d,t,i}&(放熱系統が1系統の場合)\\ 
    1&(放熱系統が2系統以上の場合)\\ 
    \end{array}
\right.
$$

$Q_{dmd,H,hs,d,t} = 0$ の場合

$$\large
r_{WS,hs,d,t} = 0
$$

$r_{WS,hs,d,t}$：日付$d$の時刻$t$における温水暖房用熱源機の温水供給運転率  
$r_{WS,ln,d,t,i}$：日付$d$の時刻$t$における放熱系統$i$の温水供給運転率（ただし、ここでは$i=1$である。）   

In [ ]:
# 温水暖房用熱源機の温水供給運転率
def get_r_WS_hs_d_t(rad_list, Q_dmd_H_hs_d_t, Q_T_H_rad, Theta_SW, region, A_A, A_MR, A_OR, mode_MR):
    # (7a)
    n = sum([rad is not None for rad in rad_list])  # 放熱系統数
    if n == 1:
        A_HCZ = get_A_HCZ_i(1, A_A, A_MR, A_OR)
        radiator = rad_list[0]
        R_type = '主たる居室'
        tmp = get_r_WS_ln_d_t_i(A_HCZ, radiator, Q_T_H_rad[0], Theta_SW, region, mode_MR, R_type)
    elif n > 1:
        tmp = np.ones(24 * 365)
    else:
        raise ValueError(n)

    # (7b)
    tmp[Q_dmd_H_hs_d_t == 0] = 0

    return tmp

#### 7.4 往き温水温度

$\theta_{SW,hs,op,1} > \theta_{SW,hs,op,2} > \theta_{SW,hs,op,3}$  

$\theta_{SW,hs,op,p}$：温水暖房用熱源機の往き温水温度の候補$p$(℃)  

<div style="text-align: center;font-weight: bold;">表　温水暖房用熱源機における往き温水温度の区分及び候補(℃)</div>

<table>
<thead>
<tr>
<th colspan="2" rowspan="2">温水暖房用熱源機の種類</th>
<th colspan="3">往き温水温度の候補の区分 $p$</th>
</tr>
<tr>
<th>1</th>
<th>2</th>
<th>3</th>
</tr>
</thead>

<tbody>
<tr>
<th rowspan="2">石油温水暖房機、石油給湯温水暖房機</th>
<th>従来型</th>
<td>60</td>
<td>-</td>
<td>-</td>
</tr>
<tr>
<th>潜熱回収型</th>
<td>60</td>
<td>40</td>
<td>-</td>
</tr>

<tr>
<th rowspan="2">ガス温水暖房機、ガス給湯温水暖房機</th>
<th>従来型</th>
<td>60</td>
<td>-</td>
<td>-</td>
</tr>
<tr>
<th>潜熱回収型</th>
<td>60</td>
<td>40</td>
<td>-</td>
</tr>

<tr>
<th colspan="2">電気ヒーター温水暖房機、電気ヒーター給湯温水暖房機</th>
<td>60</td>
<td>-</td>
<td>-</td>
</tr>

<tr>
<th colspan="2">電気ヒートポンプ温水暖房機</th>
<td>55</td>
<td>45</td>
<td>35</td>
</tr>

<tr>
<th colspan="2">電気ヒートポンプ・ガス瞬間式併用型湯温水暖房機<br>
（給湯熱源：ガス瞬間式、暖房熱源：電気ヒートポンプ・ガス瞬間式併用） </th>
<td>60</td>
<td>40</td>
<td>-</td>
</tr>

<tr>
<th colspan="2">電気ヒートポンプ・ガス瞬間式併用型給湯温水暖房機<br>
（給湯熱源：電気ヒートポンプ・ガス瞬間式併用、暖房熱源：ガス瞬間式）</th>
<td>60</td>
<td>40</td>
<td>-</td>
</tr>

<tr>
<th colspan="2">電気ヒートポンプ・ガス瞬間式併用型給湯温水暖房機<br>
（給湯熱源：電気ヒートポンプ・ガス瞬間式併用、<br>
暖房熱源：電気ヒートポンプ・ガス瞬間式併用）</th>
<td>60</td>
<td>40</td>
<td>-</td>
</tr>

<tr>
<th colspan="2">コージェネレーション設備</th>
<td colspan="3">コージェネレーション設備の種類に依る</td>
</tr>

</tbody>
</table>


In [ ]:
# 温水暖房用熱源機の往き温水温度の候補
def get_Theta_SW_hs_op(hs_type, CG_category=None):
    if hs_type == '石油従来型温水暖房機' or hs_type == '石油従来型給湯温水暖房機':
        return table_4[0]
    elif hs_type == '石油潜熱回収型温水暖房機' or hs_type == '石油潜熱回収型給湯温水暖房機':
        ## TODO: 仕様書上は40,60℃
        return (60,)
        return table_4[1]
    elif hs_type == 'ガス従来型温水暖房機' or hs_type == 'ガス従来型給湯温水暖房機' or hs_type == 'ガス従来型':
        return table_4[2]
    elif hs_type == 'ガス潜熱回収型温水暖房機' or hs_type == 'ガス潜熱回収型給湯温水暖房機' or hs_type == 'ガス潜熱回収型':
        return table_4[3]
    elif hs_type == '電気ヒーター温水暖房機' or hs_type == '電気ヒーター給湯温水暖房機':
        return table_4[4]
    elif hs_type == '電気ヒートポンプ温水暖房機':
        return table_4[5]
    elif hs_type == '電気ヒートポンプ・ガス瞬間式併用型給湯温水暖房機(給湯熱源：ガス瞬間式、暖房熱源：電気ヒートポンプ・ガス瞬間式併用)':
        return table_4[6]
    elif hs_type == '電気ヒートポンプ・ガス瞬間式併用型給湯温水暖房機(給湯熱源：電気ヒートポンプ・ガス瞬間式併用、暖房熱源：ガス瞬間式)(仕様による)':
        return table_4[7]
    elif hs_type == '電気ヒートポンプ・ガス瞬間式併用型給湯温水暖房機(給湯熱源：電気ヒートポンプ・ガス瞬間式併用、暖房熱源：電気ヒートポンプ・ガス瞬間式併用)':
        return table_4[8]
    elif hs_type == 'コージェネレーションを使用する':
        from section8_a import get_type_BB_HWH
        type_BB_HWH = get_type_BB_HWH(CG_category)
        return get_Theta_SW_hs_op(type_BB_HWH)
    elif hs_type == '給湯・温水暖房一体型を使用する':
        raise NotImplementedError()
    else:
        raise ValueError(hs_type)

$$
\large
p_{hs,d,t} = min(p_{ln,d,t,1}, p_{ln,d,t,2}, … p_{ln,d,t,m})
$$

$p_{hs,d,t}$：日付$d$の時刻$t$における温水暖房用熱源機の往き温水温度の区分(℃)  
$p_{ln,d,t,i}$：日付$d$の時刻$t$における放熱系統$i$の要求往き温水温度の区分(℃)  

In [ ]:
# 温水暖房用熱源機の往き温水温度の区分 (8)
def get_p_hs_d_t(Theta_SW_hs_op, rad_list, L_T_H_rad, A_A, A_MR, A_OR, region, mode_MR, mode_OR):
    p_ln = np.zeros((5, 24 * 365), dtype=np.int32)

    # 初期値として、最低温度を指定
    p_ln[:] = len(Theta_SW_hs_op)

    for i in [1, 3, 4, 5]:
        if rad_list[i - 1] is not None:
            A_HCZ_i = get_A_HCZ_i(i, A_A, A_MR, A_OR)
            mode = mode_MR if i == 1 else mode_OR
            R_type = '主たる居室' if i == 1 else 'その他の居室'
            p_ln_i = get_p_ln_d_t_i(rad_list[i - 1], L_T_H_rad[i - 1], Theta_SW_hs_op, A_HCZ_i, region, mode, R_type)
            p_ln[i - 1, :] = p_ln_i

    return np.min(p_ln, axis=0)


# 要求往き温水温度
def get_Theta_SW_d_t(Theta_SW_hs_op, p_hs_d_t):
    # 一括変換用のndarrayを作成
    n = len(Theta_SW_hs_op)
    array_Theta_SW_hs_op = np.array(Theta_SW_hs_op).reshape((n, 1))

    # p_d_tに基づいて1時間当たりの往き温水温度を取得
    Theta_SW_d_t = array_Theta_SW_hs_op[p_hs_d_t - 1]

    return Theta_SW_d_t.reshape(p_hs_d_t.shape)


# 表4 温水暖房用熱源機における往き温水温度の区分及び候補
table_4 = [
    (60,),
    (60, 40),
    (60,),
    (60, 40),
    (60,),
    (55, 45, 35),
    (60, 40),
    (60, 40),
    (60, 40),
]

#### 7.5 戻り温水温度

$$\large
\theta_{RW,hs,d,t} = \left\{ \begin{array}{ll}
    0.0301 \times (T_{dif})^2 - 0.1864 \times T_{dif} + 20&(熱源機の往き温水温度\theta_{SW,hs} = 60)\\ 
    0.0604 \times (T_{dif})^2 - 0.1881 \times T_{dif} + 20&(熱源機の往き温水温度\theta_{SW,hs} = 40)\\ 
    \end{array}
\right.
$$

$$\large
T_{dif,d,t} = \frac{\sum_{i}Q_{dmd,H,ln,d,t,i}}{\sum_{i}Q'_{max,rad,d,t,i} \times  0.027583}
$$

$$\large
Q'_{max,H,rad,d,t,i} = \left\{ \begin{array}{ll}
    Q_{max,H,rad,d,t,i}&(Q_{ln,dmd,d,t,i} > 0)\\ 
    0&(Q_{ln,dmd,d,t,i} = 0)\\ 
    \end{array}
\right.
$$

$T_{dif,d,t}$：日付$d$の時刻$t$における対数平均温度差(℃)  
$Q_{dmd,H,ln,d,t,i}$：日付$d$の時刻$t$における1時間当たりの放熱系統$i$の温水熱需要(MJ/h)  
$Q'_{max,H,rad,d,t,i}$：日付$d$の時刻$t$における1時間当たりの温水熱需要が発生する場合の暖冷房区画$i$に設置された放熱器の最大暖房出力(MJ/h)  
$Q_{max,H,rad,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された放熱器の最大暖房出力(MJ/h)  

In [ ]:
# 戻り温水温度 (9)
def get_Theta_RW_hs_d_t(Theta_SW_hs_d_t, rad_list, pipe_insulation, underfloor_pipe_insulation, A_A, A_MR, A_OR, region,
                        mode_MR, mode_OR,
                        L_T_H_rad):
    MR_rad_type, r_Af_1 = get_MR_rad_type_and_r_Af_1(rad_list)

    outdoor = load_outdoor()
    Theta_ex = get_Theta_ex(region, outdoor)

    Q_dmd_H_ln_d_t = np.zeros((5, 24 * 365))
    Q_dash_max_H_rad_d_t = np.zeros((5, 24 * 365))

    for i in [1, 3, 4, 5]:
        if rad_list[i - 1] is None:
            continue
        else:
            if rad_list[i - 1] is None:
                continue

            # 1時間当たりの暖冷房区画iに設置された放熱器の最大暖房出力
            A_HCZ = get_A_HCZ_i(i, A_A, A_MR, A_OR)
            R_type = '主たる居室' if i == 1 else 'その他の居室'
            mode = mode_MR if i == 1 else mode_OR
            Q_max_H_rad_d_t_i = get_Q_max_H_rad_d_t_i(rad_list[i - 1], A_HCZ, Theta_SW_hs_d_t, region, mode, R_type)

            # 1時間当たりの暖冷房区画iに設置された放熱器の処理暖房負荷
            Q_T_H_rad_d_t_i = get_Q_T_H_rad_d_t_i(Q_max_H_rad_d_t_i, L_T_H_rad[i - 1])

            # 1時間当たりの暖冷房区画iに設置された放熱器の温水供給運転率
            r_WS_rad_d_t_i = get_r_WS_rad_d_t_i(
                A_HCZ=A_HCZ,
                radiator=rad_list[i - 1],
                Q_T_H_rad=Q_T_H_rad_d_t_i,
                Theta_SW=Theta_SW_hs_d_t,
                region=region,
                mode=mode,
                R_type=R_type
            )

            Q_dmd_H_ln_d_t[i - 1] = get_Q_dmd_H_ln_d_t_i(
                i=i,
                radiator=rad_list[i - 1],
                Q_T_H_rad_d_t_i=Q_T_H_rad_d_t_i,
                Q_max_H_rad_d_t_i=Q_max_H_rad_d_t_i,
                L_T_H_rad_d_t_i=L_T_H_rad[i - 1],
                Theta_SW_d_t=Theta_SW_hs_d_t,
                Theta_ex=Theta_ex,
                r_WS_rad_d_t_i=r_WS_rad_d_t_i,
                A_A=A_A,
                pipe_insulation=pipe_insulation,
                underfloor_pipe_insulation=underfloor_pipe_insulation,
                MR_rad_type=MR_rad_type,
                r_Af_1=r_Af_1
            )

            Q_dash_max_H_rad_d_t_i = get_Q_dash_max_H_rad_d_t_i(Q_max_H_rad_d_t_i, Q_dmd_H_ln_d_t[i - 1])
            Q_dash_max_H_rad_d_t[i - 1, :] = Q_dash_max_H_rad_d_t_i

    # 1時間ごとの対数平均温度差 (9b)
    T_dif = get_T_dif_d_t(Q_dmd_H_ln_d_t, Q_dash_max_H_rad_d_t)

    # 温水暖房用熱源機の戻り温水温度 (9a)
    Theta_RW_hs_d_t = np.zeros(24 * 365)

    # - 条件1 (熱源機の往き温水温度60度)
    f1 = (Theta_SW_hs_d_t == 60)
    Theta_RW_hs_d_t[f1] = 0.0301 * T_dif[f1] ** 2 - 0.1864 * T_dif[f1] + 20

    # - 条件2 (熱源機の往き温水温度40度)
    f2 = (Theta_SW_hs_d_t == 40)
    Theta_RW_hs_d_t[f2] = 0.0604 * T_dif[f2] ** 2 - 0.1881 * T_dif[f2] + 20

    # 温水暖房用戻り温度は往き温度を超えない
    fover = (Theta_RW_hs_d_t > Theta_SW_hs_d_t)  # 温水暖房用戻り温度は往き温度を超えた日時インデックス
    Theta_RW_hs_d_t[fover] = Theta_SW_hs_d_t[fover]  # 逆転日時の限り、戻り温度=往き温度とする

    return Theta_RW_hs_d_t


# 1時間当たりの温水熱需要が発生する場合の暖冷房区画iに設置された放熱器の最大暖房出力 (9c)
def get_Q_dash_max_H_rad_d_t_i(Q_max_H_rad_d_t_i, Q_dmd_H_ln_d_t_i):
    Q_dash_max_H_rad_d_t_i = np.copy(Q_max_H_rad_d_t_i)
    Q_dash_max_H_rad_d_t_i[Q_dmd_H_ln_d_t_i == 0] = 0
    return Q_dash_max_H_rad_d_t_i


# 1時間ごとの対数平均温度差 (9b)
def get_T_dif_d_t(Q_dmd_H_ln_d_t, Q_dash_max_H_rad_d_t):
    Q_dmd = np.sum(Q_dmd_H_ln_d_t, axis=0)
    Q_dash_max = np.sum(Q_dash_max_H_rad_d_t, axis=0)

    T_dif_d_t = np.zeros(24 * 365)
    f = (Q_dmd > 0)
    T_dif_d_t[f] = Q_dmd[f] / (Q_dash_max[f] * 0.027583)

    return T_dif_d_t

### 8. 放熱系統

#### 8.1 温水熱需要

$$
\large
Q_{dmd,H,ln,d,t,i} = Q_{dmd,H,rad,d,t,i} + Q_{loss,pp,d,t,i}
$$

$Q_{dmd,H,rad,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された放熱器の温水熱需要(MJ/h)  
$Q_{loss,pp,d,t,i}$：日付$d$の時刻$t$における1時間当たりの配管$i$の熱損失(MJ/h)  

In [ ]:
# 1時間当たりの放熱系統iの温水熱需要 (10)
def get_Q_dmd_H_ln_d_t_i(i, radiator, Q_T_H_rad_d_t_i, Q_max_H_rad_d_t_i, L_T_H_rad_d_t_i, Theta_SW_d_t, Theta_ex,
                         r_WS_rad_d_t_i, A_A, pipe_insulation, underfloor_pipe_insulation,
                         MR_rad_type, r_Af_1):
    # 1時間当たりの暖冷房区画iに設置された放熱器の温水熱需要
    Q_dmd_H_rad_d_t_i = get_Q_dmd_H_rad_d_t_i(radiator, Q_max_H_rad_d_t_i, L_T_H_rad_d_t_i)

    # 1時間当たりの配管iの熱損失
    Q_loss_pp_d_t_i = get_Q_loss_pp_d_t_i(
        i=i,
        Theta_SW_d_t=Theta_SW_d_t,
        Theta_ex_d_t=Theta_ex,
        r_WS_rad_d_t_i=r_WS_rad_d_t_i,
        A_A=A_A,
        pipe_insulation=pipe_insulation,
        underfloor_pipe_insulation=underfloor_pipe_insulation,
        MR_rad_type=MR_rad_type,
        r_Af_1=r_Af_1
    )

    return Q_dmd_H_rad_d_t_i + Q_loss_pp_d_t_i

#### 8.2 温水供給運転率

$$
\large
r_{WS,ln,d,t,i} = r_{WS,rad,d,t,i}
$$

$r_{WS,rad,d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$に設置された放熱器の温水供給運転率  

In [ ]:
# 放熱系統iの温水供給運転率 (11)
def get_r_WS_ln_d_t_i(A_HCZ, radiator, Q_T_H_rad, Theta_SW, region, mode, R_type):
    return get_r_WS_rad_d_t_i(A_HCZ, radiator, Q_T_H_rad, Theta_SW, region, mode, R_type)

#### 8.3 要求往き温水温度の区分

省略

In [ ]:
# 放熱系統の要求往き温水温度の区分
def get_p_ln_d_t_i(radiator, L_T_H_rad_d_t_i, Theta_SW_hs_op, A_HCZ, region, mode, R_type):
    n = len(Theta_SW_hs_op)
    if n == 1:
        # 候補が1つの場合は決定
        return np.ones(24 * 365)
    else:
        # 候補が2つ以上の場合は未処理負荷の発生状況を確認

        # 放熱器がない暖房区画が指定されて場合は便宜上、最低の温水温度の区分を返す
        if radiator is None:
            return n * np.ones(24 * 365)

        # 往き温水温度の候補ごとに未処理負荷の発生状況を確認する
        # このとき、最大の往き温水温度の候補(p=1)は確認をしない
        flag_UT_d_t_p = np.ones((n - 1, 24 * 365))  # 基本はp=1
        for p in range(2, n + 1):
            # 往き温水温度の候補p
            Theta_SW = Theta_SW_hs_op[p - 1]

            # 往き温水温度の候補pにおける放熱器の最大出力
            Q_max_H_rad_d_t_i_p = get_Q_max_H_rad_d_t_i(radiator, A_HCZ, Theta_SW, region, mode, R_type)

            # 往き温水温度の候補pにおける1時間当たりの暖冷房区画iに設置された放熱器の処理暖房負荷
            Q_T_H_rad_d_t_i_p = get_Q_T_H_rad_d_t_i(Q_max_H_rad_d_t_i_p, L_T_H_rad_d_t_i)

            # 往き温水温度の候補pにおける1時間当たりの暖冷房区画iに設置された放熱器の未処理暖房負荷
            Q_UT_H_rad_d_t_i_p = L_T_H_rad_d_t_i - Q_T_H_rad_d_t_i_p

            # 未処理負荷が発生しなかった時刻にはpを保存
            flag_UT_d_t_i_p = Q_UT_H_rad_d_t_i_p <= 0.0
            flag_UT_d_t_p[p - 2][flag_UT_d_t_i_p] = p

        # 1時間当たりの往き温水温度の候補p
        p_ln_d_t_i = np.max(flag_UT_d_t_p, axis=0)
        return p_ln_d_t_i

### 9. 配管

$$\large
Q_{loss,pp,d,t,i} = ((\theta_{SW,d,t} - (\theta_{ex,d,t} \times 0.7 + 20 \times 0.3)) \times L_{pp,ex,i} + (\theta_{SW,d,t} - 20) \times L_{pp,in,i})\\
\large
\times K_{loss,pp,i} \times r_{WS,rad,d,t,i} \times 3600 \times 10^{-6}
$$

$\theta_{SW,d,t}$：日付$d$の時刻$t$における往き温水温度(℃)    
$\theta_{ex,d,t}$：日付$d$の時刻$t$における外気温度(℃)  
$L_{pp,ex,i}$：配管$i$の断熱区画外における長さ(m)  
$L_{pp,in,i}$：配管$i$の断熱区画内における長さ(m)  
$K_{loss,pp,i}$：配管$i$の線熱損失係数(W/mK)  
$r_{WS,rad,d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$に設置された放熱器の温水供給運転率  

In [ ]:
# 1時間当たりの配管iの熱損失 (12)
def get_Q_loss_pp_d_t_i(i, Theta_SW_d_t, Theta_ex_d_t, r_WS_rad_d_t_i, A_A, pipe_insulation, underfloor_pipe_insulation,
                        MR_rad_type, r_Af_1):
    # 配管の断熱区画外における長さ
    L_pp_ex_i = pipe.get_L_pp_ex_i(i, A_A, underfloor_pipe_insulation, MR_rad_type, r_Af_1)

    # 配管の断熱区画内における長さ
    L_pp_in_i = pipe.get_L_pp_in_i(i, A_A, underfloor_pipe_insulation, MR_rad_type, r_Af_1)

    # 線熱損失係数
    K_loss_pp_i = pipe.get_K_loss_pp(pipe_insulation)

    return ((Theta_SW_d_t - (Theta_ex_d_t * 0.7 + 20 * 0.3)) * L_pp_ex_i + (Theta_SW_d_t - 20) * L_pp_in_i) \
           * K_loss_pp_i * r_WS_rad_d_t_i * 3600 * 10 ** (-6)

### 10. 放熱器

#### 10.1 供給熱量

$$\large
Q_{dmd,H,rad,d,t,i} = Q_{T,H,rad,d,t,i} + Q_{loss,rad,d,t,i}
$$

$Q_{T,H,rad,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された放熱器の処理暖房負荷(MJ/h)  
$Q_{loss,rad,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された放熱器の熱損失(MJ/h)  

In [ ]:
# 1時間当たりの暖冷房区画iに設置された放熱器の温水熱需要 (13)
def get_Q_dmd_H_rad_d_t_i(radiator, Q_max_H_rad_d_t_i, L_T_H_rad_d_t_i):
    # 1時間当たりの暖冷房区画iに設置された放熱器の処理暖房負荷
    Q_T_H_rad_d_t_i = get_Q_T_H_rad_d_t_i(Q_max_H_rad_d_t_i, L_T_H_rad_d_t_i)

    # 1時間当たりの暖冷房区画iに設置された放熱器の熱損失
    Q_loss_rad_d_t_i = get_Q_loss_rad_d_t_i(radiator, Q_T_H_rad_d_t_i)

    return Q_T_H_rad_d_t_i + Q_loss_rad_d_t_i

#### 10.2 消費電力量

日付$d$の時刻$t$における 1 時間当たりの暖冷房区画$i$に設置された放熱器の消費電力量$E_{E,rad,d,t,i}$は、放熱器の種類がパネルラジエーター又は温水床暖房の場合は0とする。

In [ ]:
# 1時間当たりの暖冷房区画iに設置された放熱器の消費電力量
def get_E_E_rad_d_t_i(i, radiator, Q_T_H_rad, Theta_SW, A_A, A_MR, A_OR, region, mode, R_type):
    if radiator['type'] == '温水暖房用パネルラジエーター' or radiator['type'] == '温水暖房用床暖房':
        return np.zeros(24 * 365)
    elif radiator['type'] == '温水暖房用ファンコンベクター':

        A_HCZ = get_A_HCZ_i(i, A_A, A_MR, A_OR)

        return rad_fanc.get_E_E_rad(
            A_HCZ=A_HCZ,
            region=region,
            mode=mode,
            R_type=R_type,
            Theta_SW=Theta_SW,
            Q_T_H_rad=Q_T_H_rad,
        )
    else:
        raise ValueError(radiator['type'])


In [ ]:
def get_A_HCZ_i(i, A_A, A_MR, A_OR):
    return ld.get_A_HCZ_i(i, A_A, A_MR, A_OR)

#### 10.3 熱損失

日付$d$の時刻$t$における 1 時間当たりの暖冷房区画$i$に設置された放熱器の熱損失$Q_{loss,rad,d,t,i}$は、放熱器の種類がパネルラジエーター又はファンコンベクターの場合は0とする。

In [ ]:
# 1時間当たりの暖冷房区画iに設置された放熱器の熱損失
def get_Q_loss_rad_d_t_i(radiator, Q_T_H_rad):
    if radiator['type'] == '温水暖房用パネルラジエーター' or radiator['type'] == '温水暖房用ファンコンベクター':
        return np.zeros_like(Q_T_H_rad)
    elif radiator['type'] == '温水暖房用床暖房':
        return rad_floor.get_Q_loss_rad(
            r_up=radiator['r_up'],
            Q_T_H_rad=Q_T_H_rad
        )
    else:
        raise ValueError(radiator['type'])

#### 10.4 温水供給運転率

省略

#### 10.5 処理暖房負荷

省略

#### 10.6 最大暖房出力

省略